In [2]:
%load_ext sql
%sql sqlite:///PS1.db

/Users/jennifer/Library/Python/2.7/lib/python/site-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)
/Users/jennifer/Library/Python/2.7/lib/python/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")


u'Connected: None@PS1.db'

Problem Set 1
=======


### Instructions / Notes:

**_Read these carefully_**

* Run the top cell above to load the database `PS1.db` (make sure the actual database file, `PS1.db`, is in the same directory as this IPython notebook is running in)
* Some of the problems involve _changing_ this database (e.g. deleting rows)- you can always re-download `PS1.db` or make a copy if you want to start fresh!
* You **may** create new IPython notebook cells to use for e.g. testing, debugging, exploring, etc.- this is encouraged in fact!- **just make sure that your final answer for each question is _in its own cell_ and _clearly indicated_**
* When you see `In [*]:` to the left of the cell you are executing, this means that the code / query is _running_.
    * **If the cell is hanging- i.e. running for too long: To restart the SQL connection, you must restart the entire python kernel**
    * To restart kernel using the menu bar: "Kernel >> Restart >> Clear all outputs & restart"), then re-execute the sql connection cell at top
    * You will also need to restart the connection if you want to load a different version of the database file
* Remember:
    * `%sql [SQL]` is for _single line_ SQL queries
    * `%%sql [SQL]` is for _multi line_ SQL queries
* **See Piazza for submission instructions**
* _Have fun!_

Problem 1: Linear Algebra
------------------------

Two random 3x3 ($N=3$) matrices have been provided in tables `A` and `B`, having the following schema:
> * `i INT`:   Row index
> * `j INT`:   Column index
> * `val INT`: Cell value

**Note: all of your answers below _must_ work for any _square_ matrix sizes, i.e. any value of $N$**.

Note how the matrices are represented- why do we choose this format?  Run the following queries to see the matrices in a nice format:

In [ ]:
%sql SELECT group_concat(val, " , ") AS "A" FROM A GROUP BY i;

In [ ]:
%sql SELECT group_concat(val, " , ") AS "B" FROM B GROUP BY i;

### Part (a): Matrix transpose

_Transposing_ a matrix $A$ is the operation of switching its rows with its columns- written $A^T$.  For example, if we have:

$A=\begin{bmatrix}a & b & c\\ d & e & f\\ g & h & i\end{bmatrix}$

Then:

$A^T=\begin{bmatrix}a & d & g\\ b & e & h\\ c & f & i\end{bmatrix}$

Write a _single SQL query_ to get the matrix transpose $A^T$ (in the same format as $A$- output tuples should be of format `(i,j,val)` where `i` is row, `j` is column, and the output is ordered by row then column index)

Write your query here:

In [ ]:
%%sql
SELECT A.j AS i, A.i AS j, A.val
FROM A
ORDER BY i, j

In [ ]:
"""
Expected output below- don't re-evaluate this cell!

NOTE: A valid answer must work for ALL inputs of the given type,
not just this example.  I.e. do not hardcode around this answer / etc!
"""

### Part (b): Dot product I

The _dot product_ of two vectors

$a = \begin{bmatrix}a_1 & a_2 & \dots & a_n\end{bmatrix}$

and

$b = \begin{bmatrix}b_1 & b_2 & \dots & b_n\end{bmatrix}$

is

$a\cdot b = \sum_{i=1}^n a_ib_i = a_1b_1 + a_2b_2 + \dots + a_nb_n$

Write a _single SQL query_ to take the dot product of the **second column of $A$** and the **third column of $B$.**

Write your query here:

In [ ]:
%%sql
SELECT SUM(A.val * B.val)
FROM A, B
WHERE A.j = 1 AND B.j = 2 AND A.i = B.i

In [ ]:
"""
Expected output below- don't re-evaluate this cell!

NOTE: A valid answer must work for ALL inputs of the given type,
not just this example.  I.e. do not hardcode around this answer / etc!
"""

### Part (c): Dot product II

Write a _single SQL query_ to take the dot product of the **second _row_ of $A$** and the **third column of $B$.**

Write your query here:

In [ ]:
%%sql
SELECT SUM(A.val * B.val)
FROM A, B
WHERE A.i = 1 AND B.j = 2 AND A.j = B.i

In [ ]:
"""
Expected output below- don't re-evaluate this cell!

NOTE: A valid answer must work for ALL inputs of the given type,
not just this example.  I.e. do not hardcode around this answer / etc!
"""

### Part (d): Matrix multiplication

The product of a matrix $A$ (having dimensions $n\times m$) and a matrix $B$ (having dimensions $m\times p$) is the matrix $C$ (of dimension $n\times p$) having cell at row $i$ and column $j$ equal to:

$C_{ij} = \sum_{k=1}^m A_{ik}B_{kj}$

In other words, to multiply two matrices, get each cell of the resulting matrix $C$, $C_{ij}$, by taking the _dot product_ of the $i$th row of $A$ and the $j$th column of $B$.

Write a _single SQL query_ to get the matrix product of $A$ and $B$ (in the same format as $A$ and $B$).

Write your query here:

In [ ]:
%%sql
SELECT A.i, B.j, SUM(A.val * B.val) AS val
FROM A, B
WHERE A.j = B.i
GROUP BY A.i, B.j

In [ ]:
"""
Expected output below- don't re-evaluate this cell!

NOTE: A valid answer must work for ALL inputs of the given type,
not just this example.  I.e. do not hardcode around this answer / etc!
"""

Problem 2: U.S. Hourly NOAA Precipitation dataset
----------------------------------------------

We've prepared and loaded a [public dataset](https://catalog.data.gov/dataset/u-s-hourly-precipitation-data) from the US NOAA (National Oceanic and Atmospheric Administration) of daily precipitation data from weather stations across CA from the month of Sep. 2013.  We'll use the `precipitation` table here, which has a simplified schema:

> `station_id INT`

> `day INT`

> `precipitation INT`

In [ ]:
%sql SELECT * FROM precipitation LIMIT 3;

### Part (a): State Champions

Using a _single SQL query_, find all of the stations that had the highest daily precipitation (across all CA stations) on any given day **more than once**, and return the counts of how many "best days" they had in descending order.  Further requirements:
* Use `GROUP BY`
* Write the shortest possible SQL query to accomplish this
* Return relation `(station_id, num_best_days)`

*Hint: Make sure your query correctly handles ties!*

Write your query here:

In [ ]:
%%sql
SELECT p.station_id, COUNT(p.station_id) AS num_best_days
FROM (
    SELECT day, MAX(precipitation) AS precipitation
    FROM precipitation
    GROUP BY day
    ) AS pmax, precipitation p
WHERE p.day = pmax.day AND p.precipitation = pmax.precipitation
GROUP BY station_id
HAVING COUNT(p.station_id) > 1
ORDER BY num_best_days DESC


In [ ]:
"""
Expected output below- don't re-evaluate this cell!

NOTE: A valid answer must work for ALL inputs of the given type,
not just this example.  I.e. do not hardcode around this answer / etc!
"""

### Part (b.i): Median value, Pt. I

Our goal in this part is going to be to find the **median value** of a list of values.  We want to do this for the general case, however we'll start with a slightly simplified setting:

Write a _single SQL query_ to find the median value of a certain attribute in a table, given that:
* The table contains an odd number of rows
* The values of this attribute are unique in the table (e.g. no two rows have the same value for this attribute)

Also, **do not hard code the size of the table and/or use any `ORDER BY` clause.  Think about the definition of median!**

Let's test using the table $X$, constructed out of the distinct precipitation values of all stations on all days, having one attribute $p$:

In [ ]:
%%sql
DROP TABLE IF EXISTS X;
CREATE TABLE X AS SELECT DISTINCT precipitation AS p FROM precipitation;
SELECT COUNT(*) FROM X;

Write your query here:

In [ ]:
%%sql
SELECT x1.p AS median
FROM X AS x1
WHERE (SELECT COUNT(*) FROM X AS x2 WHERE x1.p > x2.p) 
     =
      (SELECT COUNT(*) FROM X AS x2 WHERE x1.p < x2.p)

In [ ]:
"""
Expected output below- don't re-evaluate this cell!

NOTE: A valid answer must work for ALL inputs of the given type,
not just this example.  I.e. do not hardcode around this answer / etc!
"""

### Part (b.ii): Median value, Pt. II

Now, we want to find the median precipitation value across all days for a station `376101`.  **However to get credit, your query must work for ALL stations- test it on some others to be certain it's correct!**

Write a _single SQL query_ to find the median.  Again, do not hardcode the size of any table and/or use any `ORDER BY` clause.

Note that now:
* The number of rows can be even
* The values can have duplicates

Also note that we will use the definition of _median_ where ties (e.g. when there are an even number of rows) are broken by averaging.  Write your query here:

In [ ]:
%%sql
-- Wrong answer, note that there are duplicates
SELECT p1.precipitation
FROM precipitation p1
WHERE p1.station_id = 376101 
    AND (SELECT COUNT(*) 
         FROM precipitation AS p2 
         WHERE p2.station_id = 376101 AND p1.precipitation >= p2.precipitation)
        >=
        (SELECT COUNT(*) 
         FROM precipitation AS p2 
         WHERE p2.station_id = 376101 AND p1.precipitation < p2.precipitation)
    AND (SELECT COUNT(*) 
         FROM precipitation AS p2 
         WHERE p2.station_id = 376101 AND p1.precipitation > p2.precipitation)
        <=
        (SELECT COUNT(*) 
         FROM precipitation AS p2
         WHERE p2.station_id = 376101 AND p1.precipitation <= p2.precipitation)


In [ ]:
%%sql
SELECT AVG(p.precipitation)
FROM (
    SELECT p1.precipitation
    FROM precipitation AS p1
    WHERE p1.station_id = 376101 
        AND (SELECT COUNT(*) 
             FROM precipitation AS p2 
             WHERE p2.station_id = 376101 AND p1.precipitation >= p2.precipitation)
            >=
            (SELECT COUNT(*) 
             FROM precipitation AS p2 
             WHERE p2.station_id = 376101 AND p1.precipitation < p2.precipitation)
        AND (SELECT COUNT(*) 
             FROM precipitation AS p2 
             WHERE p2.station_id = 376101 AND p1.precipitation > p2.precipitation)
            <=
            (SELECT COUNT(*) 
             FROM precipitation AS p2
             WHERE p2.station_id = 376101 AND p1.precipitation <= p2.precipitation)
    GROUP BY precipitation
    ) AS p


In [ ]:
"""
Expected output below- don't re-evaluate this cell!

NOTE: A valid answer must work for ALL inputs of the given type,
not just this example.  I.e. do not hardcode around this answer / etc!
"""

### Part (c):

Write a _single SQL query_ to find stations in CA which had a _smallest rainy day precipitation value_ (i.e. smallest non-zero `precipitation`) that was **within 400** of the _largest overall precipitation value_ (across all stations & all days).  Return tuples of type `(station_id, min_rainy_day_precip)`.

*Note: do not hard-code the maximum daily precipitation value, or any other values.*

**Do this using `GROUP BY` and aggregate functions (e.g. `COUNT`, `SUM`, `MAX`, `MIN`)**.  Write your query here:

In [ ]:
%%sql
SELECT p1.station_id, p1.min_daily
FROM (
    SELECT station_id, MIN(precipitation) AS min_daily
    FROM precipitation AS p
    WHERE p.precipitation > 0
    GROUP BY p.station_id
    ) AS p1
WHERE p1.min_daily > (SELECT MAX(precipitation) FROM precipitation) - 400

In [ ]:
"""
Expected output below- don't re-evaluate this cell!

NOTE: A valid answer must work for ALL inputs of the given type,
not just this example.  I.e. do not hardcode around this answer / etc!
"""

### Part (d):

Do the same as above, except do **not** use `GROUP BY` or any aggregate functions.  Write your query here:

In [ ]:
%%sql
SELECT p.station_id, p.min_daily
FROM (
    SELECT p1.station_id, p1.precipitation AS min_daily
    FROM precipitation p1
    WHERE p1.precipitation > 0 AND
        NOT EXISTS (
            SELECT p2.precipitation
            FROM precipitation p2 
            WHERE p2.station_id = p1.station_id 
                AND p2.precipitation < p1.precipitation
                AND p2.precipitation > 0
        )
    ) AS p
WHERE NOT EXISTS (
    SELECT *
    FROM precipitation p3
    WHERE p.min_daily + 400 < p3.precipitation 
    ) 

In [ ]:
"""
Expected output below- don't re-evaluate this cell!

NOTE: A valid answer must work for ALL inputs of the given type,
not just this example.  I.e. do not hardcode around this answer / etc!
"""

Problem 3: The Traveling SQL Server Salesman Problem
--------------------------------------------------

SQL Server salespeople are lucky as far as traveling salespeople go- they only have to sell one or two big enterprise contracts, at one or two offices in Silicon Valley, in order to make their monthly quota!

Answer the following questions using the table of streets connecting company office buildings.

**Note that for convenience all streets are included _twice_, as $A \rightarrow B$ and $B \rightarrow A$.  This should make some parts of the problem easier, but remember to take it into account!**

In [ ]:
%sql SELECT * FROM streets LIMIT 4;

### Part (a): One-hop, two-hop, three-hop...

Our salesperson has stopped at Stanford, to steal some cool new RDBMS technology from CS145-ers, and now wants to go sell it to a company _within 10 miles of Stanford_ and _passing through no more than 3 distinct streets_.  Write a single query, not using `WITH` (see later on), to find all such companies.

Your query should return tuples `(company, distance)` where distance is cumulative from Stanford.

Write your query here:

In [ ]:
%%sql
-- Not the best way
SELECT B, d
FROM streets
WHERE A = 'Stanford'
AND d < 10
UNION
SELECT e.B, (e.d + s.d)
FROM streets AS e,
    (SELECT A, B, d
    FROM streets
    WHERE A = 'Stanford'
    AND d < 10
    ) AS s
WHERE s.B = e.A AND (e.d + s.d) < 10
-- UNION
-- ...

In [ ]:
%%sql
SELECT B AS company, d AS distance
FROM (
    SELECT s1.B, s1.d FROM streets AS s1 WHERE s1.d <= 10 AND s1.A = 'Stanford'
    UNION
    SELECT s2.B, (s2.d + s1.d) AS d FROM streets s1, streets s2 WHERE (s2.d + s1.d) <= 10
        AND s1.A = 'Stanford' AND s1.B = s2.A AND s2.B <> s1.A
    UNION
    SELECT s3.B, (s2.d + s1.d + s3.d) AS d FROM streets s1, streets s2, streets s3
        WHERE (s2.d + s1.d + s3.d) <= 10 AND s1.A = 'Stanford' AND s1.B = s2.A AND s2.B = s3.A
        AND s2.B <> s1.A AND s3.B <> s2.A AND s3.B <> s1.A
)

In [ ]:
"""
Expected output below- don't re-evaluate this cell!

NOTE: A valid answer must work for ALL inputs of the given type,
not just this example.  I.e. do not hardcode around this answer / etc!
"""

### Part (b): A stop at the Farm

Now, our salesperson is out in the field, and wants to see all routes- and their distances- which will take him/her from a company $A$ to a company $B$, with the following constraints:
* The route must _pass through Stanford_ (in order to pick up new RDBMS tech to sell!)
* $A$ and $B$ must _each individually_ be _within 3 hops of Stanford_
* $A$ and $B$ must be different companies
* _The total distance must be $<= 15$_
* Do not use `WITH`
* If you return a path $A \rightarrow B$, _also include $B \rightarrow A$ in your answer_

In order to make your answer a bit cleaner, you may split into two queries, one of which creates a `VIEW`.  A view is a virtual table based on the output set of a SQL query.  A view can be used just like a normal table- the only difference under the hood is that the DBMS re-evaluates the query used to generate it each time a view is queried by a user (thus the data is always up-to date!)

Here's a simple example of a view:

In [ ]:
%%sql 
DROP VIEW IF EXISTS short_streets;
CREATE VIEW short_streets AS 
SELECT A, B, d FROM streets WHERE d < 3;
SELECT * FROM short_streets LIMIT 3;

Write your query or queries here:

In [ ]:
%%sql
DROP VIEW IF EXISTS non_cyclic_distance;
CREATE VIEW non_cyclic_distance AS
SELECT s1.A, s1.B, s1.d AS dist
FROM streets s1
UNION
SELECT s1.A, s2.B, (s1.d + s2.d) AS dist
FROM streets s1, streets s2
WHERE s1.B = s2.A AND s2.B <> s1.A 
UNION
SELECT s1.A, s3.B, (s1.d + s2.d + s3.d) AS dist
FROM streets s1, streets s2, streets s3
WHERE s1.B = s2.A AND s2.B = s3.A AND s2.B <> s1.A AND s3.B <> s2.A AND s3.B <> s1.A;

SELECT d1.B AS company_1, d2.B AS company_2, MIN(d1.dist + d2.dist) as distance
FROM non_cyclic_distance d1, non_cyclic_distance d2
WHERE d1.A = 'Stanford' AND d2.A = 'Stanford' AND d1.B <> d2.B
GROUP BY d1.B, d2.B
HAVING distance <= 15 -- interesting!

In [ ]:
"""
Expected output below- don't re-evaluate this cell!

NOTE: A valid answer must work for ALL inputs of the given type,
not just this example.  I.e. do not hardcode around this answer / etc!
"""

### Part (c): Ensuring acyclicity

You may have noticed at this point that the network, or **_graph_**, of streets in our `streets` table seems like it might be a **tree**.

Recall that a **_tree_** is an undirected graph where each node has exactly one parent (or, is the root, and has none), but may have multiple children.  A slightly more formal definition of a tree is as follows: 

> An undirected graph $T$ is a **_tree_** if it is **connected**- meaning that there is a path between every pair of nodes- and has no **cycles** (informally, closed loops)

Suppose that we guarantee the following about the graph of companies & streets determined by the `streets` table:
* It is _connected_
* It has no cycles of length $> 3$

Write a _single SQL query_ which, if our graph is not a tree (i.e. if this isn't a [shaggy dog problem](https://en.wikipedia.org/wiki/Shaggy_dog_story)), **turns it into a tree** by deleting exactly _one_ street (*= two entries in our table!*).  Write your query here:

In [ ]:
%reload_ext sql
%sql sqlite:///PS1.db

In [ ]:
%%sql
DELETE FROM streets
WHERE id IN
(
    SELECT s1.id
    FROM streets s1, streets s2, streets s3
    WHERE s1.B = s2.A AND s2.B = s3.A AND s3.B = s1.A
    LIMIT 1 -- Because these three companies form a cycle
);

In [ ]:
"""
Expected output below- don't re-evaluate this cell!

NOTE: A valid answer must work for ALL inputs of the given type,
not just this example.  I.e. do not hardcode around this answer / etc!
"""

### Part (d): The longest path

In this part, we want to find the distance of the _longest path between any two companies_.

Note that you should probably have done Part (c) first so that the graph of streets is a _tree_- this will make it much easier to work with!

If you've done the other parts above, you might be skeptical that SQL can find paths of arbitrary lengths (which is what we need to do for this problem); how can we do this?

There are some non-standard SQL functions- i.e. not universally supported by all SQL DBMSs- which are often incredibly useful.  One of these is the `WITH RECURSIVE` clause, supported by SQLite.

A `WITH` clause lets you define what is essentially a view within a clause, mainly to clean up your SQL code.  A trivial example, to illustrate `WITH`:

In [ ]:
%%sql
WITH companies(name) AS (
    SELECT DISTINCT A FROM streets)
SELECT * 
FROM companies 
WHERE name LIKE '%Gizmo%';

There is also a recursive variant, `WITH RECURSIVE`.  `WITH RECURSIVE` allows you to define a view just as above, except the table can be defined recursively.  A `WITH RECURSIVE` clause must be of the following form:

```
WITH RECURSIVE 
    R(...) AS (
        SELECT ... 
        UNION [ALL] 
        SELECT ... FROM R, ...
    )
...
```

`R` is the _recursive table_.  The `AS` clause contains two `SELECT` statements, conjoined by a `UNION` or `UNION ALL`; the first `SELECT` statement provides the initial / base case values, and the second or _recursive_ `SELECT` statement must include the recursive table in its `FROM` clause.

Basically, the recursive `SELECT` statement continues executing on the tuple _most recently inserted into `R`_, inserting output rows back into `R`, and proceeding recursively in this way, until it no longer outputs any rows, and then stops.  See the [SQLite documentation](https://www.sqlite.org/lang_with.html) for more detail.

The following example computes $5! = 5*4*3*2*1$ using `WITH RECURSIVE`:

In [ ]:
%%sql
WITH RECURSIVE
    factorials(n,x) AS (
        SELECT 1, 1
        UNION
        SELECT n+1, (n+1)*x FROM factorials WHERE n < 5)
SELECT x FROM factorials WHERE n = 5;

In this example:

1. First, `(1,1)` is inserted into the table `factorials` (the base case).
2. Next, this tuple is returned by the recursive select, as `(n, x)`, and we insert the result back into `factorials`: `(1+1, (1+1)*1) = (2,2)`
3. Next, we do the same with the last tuple inserted into `factorials`- `(2,2)`- and insert `(2+1, (2+1)*2) = (3,6)`
4. And again: get `(3,6)` from `factorials` and insert `(3+1, (3+1)*6) = (4,24)` back in
5. And again: `(4,24)` -> `(4+1, (4+1)*24) = (5,120)`
6. Now the last tuple inserted into `factorials` is `(5, 120)`, which fails the `WHERE n < 5` clause, and thus our recursive select returns an empty set, and our `WITH RECURSIVE` statement concludes
7. Finally, now that our `WITH RECURSIVE` clause has concluded, we move on to the `SELECT x FROM factorials WHERE n=5` clause, which gets us our answer!

#### Now, your turn!

Write a single SQL query that uses `WITH RECURSIVE` to find the furthest (by distance) pair of companies that still have a path of streets connecting them.  Your query should return `(A, B, distance)`.

Write your query here:

In [4]:
%%sql
SELECT A, B, A, d
FROM streets
LIMIT 1

Done.


A,B,A_1,d
Stanford,DooHickey Collective,Stanford,7


In [7]:
%%sql
WITH RECURSIVE
    paths(A, B, prev, d) AS (
        SELECT A, B, A, d
        FROM streets
        UNION ALL
        SELECT p.A, s.B, s.A, s.d + p.d
        FROM streets s, paths p
        WHERE p.B = s.A AND s.B <> p.A AND s.b <> p.prev
    )
SELECT A, B, MAX(d) FROM paths;

Done.


A,B,MAX(d)
GadgetWorks,ThingWorks,63


In [ ]:
"""
Expected output below- don't re-evaluate this cell!

NOTE: A valid answer must work for ALL inputs of the given type,
not just this example.  I.e. do not hardcode around this answer / etc!
"""

### Note on alternate output

**NOTE:** The **_distance_** of the longest path could be **49 _OR_ 63** depending on which street you deleted in Part (c)!

### Bonus Problem 1: The longest path, Pt. II

Using `WITH RECURSIVE` may be a little tricky syntactically, but it is quite elegant.  What would alternatives look like?  We already know we can't do it with non-recursive SQL.

For this problem, **use SQL _and_ Python** to find the longest path (by cumulative distance) between two companies in the streets graph, **without using a `WITH` clause**, again returning a single tuple of the form `(A, B, distance)`.

_Note: Be careful of trivial cycles in the graph!  Especially if you write recursive functions in your python code, note that IPython handles hitting the max recursion depth pretty poorly (i.e. crashes / freezes up).  You can debug in a normal terminal first if this is a concern._

Write your code / queries here:

In [ ]:
"""
Expected output below- don't re-evaluate this cell!

NOTE: A valid answer must work for ALL inputs of the given type,
not just this example.  I.e. do not hardcode around this answer / etc!
"""

### Note on alternate output

**NOTE:** The **_distance_** of the longest path could be **49 _OR_ 63** depending on which street you deleted in Part (c)!

Bonus Problem 2: How many cooks?
-------------------------------

What does the following query get you?  Think about it before you execute it!

In [ ]:
%%sql
WITH RECURSIVE
    cooks(x) AS (SELECT 1 UNION ALL SELECT x+1 FROM cooks)
SELECT x FROM cooks;

[Hint](https://www.youtube.com/watch?v=QrGrOK8oZG8)